In [15]:
from azure.storage.blob import BlobServiceClient
import json
import pandas as pd
from datetime import datetime

# Connect to Azure Blob Storage
connection_string = "DefaultEndpointsProtocol=https;AccountName=sg092620240215;AccountKey=+PaTF6WCZ0NY63Hni1XIWRJfWsnTI7QJCLVP0f1OXUoVzJyl0AcE4h2Pe1b7ZbgldGkDDFA0j9iK+AStvU4auA==;EndpointSuffix=core.windows.net"
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
bronze_client = blob_service_client.get_container_client("bronze")
silver_client = blob_service_client.get_container_client("silver")

# Load all_players from bronze/players_data.json
blob_name = "players_data.json"
print(f"Loading {blob_name} from bronze container")
blob_client = bronze_client.get_blob_client(blob_name)
blob_data = blob_client.download_blob().readall()
all_players = json.loads(blob_data.decode("utf-8"))

# Convert to DataFrame and then CSV
if not all_players:
    raise ValueError("No player data found in players_data.json")
try:
    df = pd.DataFrame(all_players)
    csv_data = df.to_csv(index=False)  # Flatten to CSV without index
except Exception as e:
    raise ValueError(f"Failed to convert to CSV: {str(e)}")

print(csv_data)


Loading players_data.json from bronze container
pos,name,id,ADP
DH,"Ohtani, Shohei",02yc4,1.34
DH,"Ohtani-H, Shohei",06alt,1.91
SS,"Witt Jr., Bobby",04lgp,2.37
CF,"Judge, Aaron",02z0s,2.67
3B,"Ramirez, Jose",01ub6,5.34
SS,"De La Cruz, Elly",05udp,5.48
RF,"Soto, Juan",03ynj,5.53
SS,"Henderson, Gunnar",050w9,7.0
RF,"Tucker, Kyle",03pir,8.41
RF,"Betts, Mookie",02btf,10.89
CF,"Carroll, Corbin",04y4e,11.29
SP,"Skubal, Tarik",04ptt,12.37
SS,"Lindor, Francisco",02611,12.87
RF,"Tatis Jr., Fernando",03zbh,13.64
SP,"Skenes, Paul",05wru,13.76
1B,"Guerrero Jr., Vladimir",03ojr,14.58
CF,"Rodriguez, Julio",04fhb,16.52
LF,"Alvarez, Yordan",04355,17.93
SP,"Wheeler, Zack",01sb3,18.93
1B,"Harper, Bryce",01fh9,20.63
LF,"Chourio, Jackson",05m1e,21.87
CF,"Duran, Jarren",04pst,23.0
RF,"Acuna Jr., Ronald",03wha,24.68
2B,"Marte, Ketel",0326s,25.47
1B,"Freeman, Freddie",01cgo,26.63
SS,"Turner, Trea",02c68,27.62
LF,"Schwarber, Kyle",03cag,28.27
3B,"Devers, Rafael",030uc,28.84
SP,"Gilbert, Logan",04n0m,29.21
1B,

In [16]:

# Save to silver container as CSV with timestamp
output_blob_name = f"all_players_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
output_blob_client = silver_client.get_blob_client(output_blob_name)
output_blob_client.upload_blob(csv_data, overwrite=True)
print(f"Uploaded {output_blob_name} to silver container")

Uploaded all_players_20250305_145657.csv to silver container
